<a href="https://colab.research.google.com/github/Moukthika1253/Reddit-comments-text-classifier/blob/main/SubReddit_category_text_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip /content/archive.zip

Archive:  /content/archive.zip
  inflating: Top_Posts.csv           
  inflating: Top_Posts_Comments.csv  


**Loading dataset**

In [5]:
import pandas as pd
post_df=pd.read_csv('/content/Top_Posts.csv')
comment_df=pd.read_csv('/content/Top_Posts_Comments.csv')

In [6]:
post_df.head()

,post_id,post_title,subreddit,post_url,flair_text,score,comments,upvote_ratio,date-time,year
0,gh1dj9,[Project] From books to presentations in 10s w...,MachineLearning,https://v.redd.it/v492uoheuxx41,Project,7798,186,0.99,2020-05-10 13:19:54,2020
1,kuc6tz,[D] A Demo from 1993 of 32-year-old Yann LeCun...,MachineLearning,https://v.redd.it/25nxi9ojfha61,Discussion,5851,133,0.98,2021-01-10 10:30:36,2021
2,g7nfvb,[R] First Order Motion Model applied to animat...,MachineLearning,https://v.redd.it/rlmmjm1q5wu41,Research,4761,111,0.97,2020-04-25 04:27:23,2020
3,lui92h,[N] AI can turn old photos into moving Images ...,MachineLearning,https://v.redd.it/ikd5gjlbi8k61,News,4688,230,0.97,2021-02-28 15:12:28,2021
4,ohxnts,[D] This AI reveals how much time politicians ...,MachineLearning,https://i.redd.it/34sgziebfia71.jpg,Discussion,4568,228,0.96,2021-07-11 04:18:59,2021


In [7]:
comment_df.head()

,post_id,comment
0,gh1dj9,Twitter thread: [https://twitter.com/cyrildiag...
1,gh1dj9,The future 🤯
2,gh1dj9,Simple yet very useful. Thank you for sharing ...
3,gh1dj9,"Almost guaranteed, Apple will copy your idea i..."
4,gh1dj9,Ohh the nightmare of making this into a stable...


In [9]:
post_comment_df=pd.merge(post_df,comment_df,how='inner',on='post_id')
post_comment_df.head()

,post_id,post_title,subreddit,post_url,flair_text,score,comments,upvote_ratio,date-time,year,comment
0,gh1dj9,[Project] From books to presentations in 10s w...,MachineLearning,https://v.redd.it/v492uoheuxx41,Project,7798,186,0.99,2020-05-10 13:19:54,2020,Twitter thread: [https://twitter.com/cyrildiag...
1,gh1dj9,[Project] From books to presentations in 10s w...,MachineLearning,https://v.redd.it/v492uoheuxx41,Project,7798,186,0.99,2020-05-10 13:19:54,2020,The future 🤯
2,gh1dj9,[Project] From books to presentations in 10s w...,MachineLearning,https://v.redd.it/v492uoheuxx41,Project,7798,186,0.99,2020-05-10 13:19:54,2020,Simple yet very useful. Thank you for sharing ...
3,gh1dj9,[Project] From books to presentations in 10s w...,MachineLearning,https://v.redd.it/v492uoheuxx41,Project,7798,186,0.99,2020-05-10 13:19:54,2020,"Almost guaranteed, Apple will copy your idea i..."
4,gh1dj9,[Project] From books to presentations in 10s w...,MachineLearning,https://v.redd.it/v492uoheuxx41,Project,7798,186,0.99,2020-05-10 13:19:54,2020,Ohh the nightmare of making this into a stable...


In [17]:
target_df=post_comment_df[['post_id','subreddit','comment']]
target_df.head()

,post_id,subreddit,comment
0,gh1dj9,MachineLearning,Twitter thread: [https://twitter.com/cyrildiag...
1,gh1dj9,MachineLearning,The future 🤯
2,gh1dj9,MachineLearning,Simple yet very useful. Thank you for sharing ...
3,gh1dj9,MachineLearning,"Almost guaranteed, Apple will copy your idea i..."
4,gh1dj9,MachineLearning,Ohh the nightmare of making this into a stable...


**Split into Train, Dev, Test dataset**

In [22]:
from sklearn.model_selection import train_test_split
X_train,X_remaining, y_train, y_remaining= train_test_split(target_df['comment'],target_df['subreddit'],train_size=0.5,stratify=target_df['subreddit'])
X_test, X_dev, y_test, y_dev = train_test_split(X_remaining,y_remaining,test_size=0.3,stratify=y_remaining)
train_set=pd.concat([X_train,y_train],axis=1).reset_index(drop=True)
dev_set=pd.concat([X_dev,y_dev],axis=1).reset_index(drop=True)
test_set=pd.concat([X_test,y_test],axis=1).reset_index(drop=True)
print(f"Train set size:  ",train_set.size)
print(f"Dev set size:  ",dev_set.size)
print(f"Test set size:  ",test_set.size)

Train set size:   223168
Dev set size:   66952
Test set size:   156216


In [24]:
train_set.head()

,comment,subreddit
0,Are you make this graphics by yourself ? I am...,artificial
1,So is SQL like dplyr but more efficient?,datascience
2,> I made it from college with no internship ->...,datascience
3,I don’t have an ax to grind.\n\nJust in genera...,MachineLearning
4,Sociology degree here. I had to take more stat...,datascience


In [36]:
import numpy as np
train_set.isna().sum()
train_set['comment'].replace('',np.nan,inplace=True)
train_set.dropna(subset=['comment'],inplace=True)

In [37]:
train_set.isna().sum()

comment      0
subreddit    0
dtype: int64

In [38]:
train_copy=train_set.copy()
train_copy.head()

,comment,subreddit
0,Are you make this graphics by yourself ? I am...,artificial
1,So is SQL like dplyr but more efficient?,datascience
2,> I made it from college with no internship ->...,datascience
3,I don’t have an ax to grind.\n\nJust in genera...,MachineLearning
4,Sociology degree here. I had to take more stat...,datascience


**Pre-processing**

In [39]:
import string,re
import nltk
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
def text_preprocess(comment):
  tokens=[word for word in word_tokenize(comment)]
  tokens1=[word.lower() for word in tokens if word.lower() not in stopwords.words('english')]
  for i in range(len(tokens1)):
   tokens1[i]= ''.join([word for word in tokens1[i] if word not in string.punctuation])
  tokens2=[word for word in tokens1 if word]
  tokens3=[re.sub(r'\s\s+','',word) for word in tokens2]
  word_lem=WordNetLemmatizer()
  lem_words=[word_lem.lemmatize(word) for word in tokens3]
  return lem_words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
train_copy['comment']=train_copy['comment'].apply(text_preprocess)
train_copy.head()

,comment,subreddit
0,"[make, graphic, really, impressed, post, also,...",artificial
1,"[sql, like, dplyr, efficient]",datascience
2,"[made, college, internship, data, analyst, dat...",datascience
3,"[’, ax, grind, general, ml, researcher, opinio...",MachineLearning
4,"[sociology, degree, take, stats, course, get, ...",datascience
